In [1]:
import os, sys
import time
# set the working directory
sys.path.append(os.getcwd()+ "\..\..")

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pycaret.classification as pc
import seaborn as sns
from imblearn.over_sampling import SMOTEN, ADASYN
from imblearn.under_sampling import RandomUnderSampler
from sklearn import preprocessing
from sklearn.ensemble import ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, precision_score, recall_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

# import from ../script/data.py in scripts folder
from src.scripts.data import CR8_smoke_dict, OR45_dict 
from skopt import BayesSearchCV
from skopt.space import Real, Integer, Categorical

In [32]:
def read_dataframes(folder_path):
    file_list = [f for f in os.listdir(folder_path) if f.endswith('.sas7bdat')]
    print(file_list)
    dataframes = {file: pd.read_sas(os.path.join(folder_path, file)) for file in file_list}
    return dataframes


def get_common_columns(dataframes):
    # rename all columns to lower case
    for df in dataframes.values():
        df.columns = df.columns.str.upper()
    
    common_columns = dataframes[list(dataframes.keys())[0]].columns
    for df in dataframes.values():
        common_columns = common_columns.intersection(df.columns)
    return common_columns


def preprocess_dataframes(dataframes, common_columns):
    merged_df = pd.DataFrame()
    for i, (file, df) in enumerate(dataframes.items()):
        df = df[common_columns]
        # df.insert(0, "State", file.split(" ")[2])
        df.insert(0, "State", i)
        merged_df = pd.concat([merged_df, df])
    return merged_df

Data preprocessing

In [33]:
# open sas7bdat file
# file = pd.read_sas('../../data/raw/GATS/GATS_Greece_National_2013_SAS/GREECE_PUBLIC_USE_11Mar2015.sas7bdat')

# Read dataframes from GYTS folder
dataframes = read_dataframes("../../data/raw/GATS/")

# Find common columns in all dataframes
common_columns = get_common_columns(dataframes)

# Preprocess dataframes
merged_df = preprocess_dataframes(dataframes, common_columns)

# Drop rows with missing values
# merged_df = merged_df.dropna()

['GATS 2011 Poland.sas7bdat', 'GATS 2015 Greece.sas7bdat', 'GATS 2018 Romania.sas7bdat']


In [ ]:
merged_df.rename(columns={"AGE": "Age"}, inplace=True)
merged_df.rename(columns={"A01": "Gender"}, inplace=True)
merged_df.rename(columns={"HH4E": "Smoke"}, inplace=True)
merged_df.rename(columns={"A04": "Education"}, inplace=True)

# 1 no formal education
# 2 primary education
# 3 high school
# 4 secondary school
# 5 university
# 6 postgraduate
# 7 other

greece_dict = {1:1, 2:1, 3:2, }
poland_dict = {1:1, 2:1, 3:2}
romania_dict = {1:1,2:2,  }
